In [1]:
import pandas as pd
from evaluate import load

/home/ubuntu/Projects/audio-benchmark/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wer = load("wer")

In [22]:
files = ['audios', 'gcp-stt', 'oci_speech', 'whisper-large-v3-turbo']
df_audios, df_stt, df_whisper, df_oci = (pd.read_csv(f"data/{f}.csv") for f in files)

In [25]:
df_audios.head()

,path,sentence,duration,sr
0,data/audio/00010.mp3,Já é uma sorte que você é poupado de um acidente.,9.648,48000
1,data/audio/00076.mp3,"esquadrão da morte, jogo do bicho, máfia do ap...",9.756,48000
2,data/audio/00083.mp3,"burnout, anorexia nervosa, bulimia, periódica,...",9.972,48000
3,data/audio/00084.mp3,Duas mulheres e uma menina levantam com troféus.,10.176,48000
4,data/audio/00092.mp3,Mesa de sinuca no bar,10.620,48000


In [26]:
df_stt.head()

,file,transcription,total_seconds
0,00010.mp3,já é uma sorte que você é poupado de um acidente,7.97
1,00076.mp3,Esquadrão da Morte jogo do bicho máfia do apit...,9.19
2,00083.mp3,marxia nervosa bulimia perodica alimentar a co...,9.91
3,00084.mp3,duas mulheres em uma menina levantam com troféus,6.32
4,00092.mp3,mesa de sinuca no par,6.33


In [27]:
df_whisper.head()

,file,transcription,total_seconds
0,00010.mp3,Já é uma sorte você é poupado de um acidente.,17
1,00076.mp3,"Esquadrão da Morte, Jogo do Bicho, Máfia do Ap...",17
2,00083.mp3,"Pernalte, anorexia nervosa, bulimia peróidica,...",21
3,00084.mp3,Duas mulheres e uma menina levantam com troféus.,15
4,00092.mp3,Mesmo de sinuca no parque.,21


In [28]:
df_oci.head()

,file,transcription,total_seconds
0,00010.mp3,já é uma sorte que você é poupado de um acidente,0.673280
1,00076.mp3,"Esquadrão da Morte, Jogo do Bicho, Máfia do A...",0.285897
2,00083.mp3,"...tornal, temorexia nervosa, bulimia periódi...",0.316535
3,00084.mp3,Duas mulheres e uma menina levantam com troféus.,0.185692
4,00092.mp3,Mesa de sinuca no bar.,0.143915


In [38]:
wer.compute(predictions=df_audios.sentence, references=df_stt.transcription)

0.47752808988764045

In [39]:
wer.compute(predictions=df_audios.sentence, references=df_whisper.transcription)

0.2593283582089552

In [40]:
wer.compute(predictions=df_audios.sentence, references=df_oci.transcription)

0.24442379182156135